In [1]:
!cp /content/drive/"My Drive"/kaggle/moa/lish-moa.zip /content/
!unzip lish-moa.zip


Archive:  lish-moa.zip
  inflating: sample_submission.csv   
  inflating: test_features.csv       
  inflating: train_features.csv      
  inflating: train_targets_nonscored.csv  
  inflating: train_targets_scored.csv  


In [2]:
!cp /content/drive/"My Drive"/kaggle/moa//models.zip /content/
!unzip models.zip

Archive:  models.zip
   creating: models/
  inflating: models/model_1.pth      
  inflating: models/model_3.pth      
  inflating: models/model_2.pth      
  inflating: models/model_4.pth      
  inflating: models/model_5.pth      
  inflating: models/model_6.pth      
  inflating: models/model_7.pth      
  inflating: models/model_8.pth      
  inflating: models/model_9.pth      
  inflating: models/model_108.pth    
  inflating: models/model_103.pth    
  inflating: models/model_10.pth     
  inflating: models/model_106.pth    
  inflating: models/model_107.pth    
  inflating: models/model_102.pth    
  inflating: models/model_104.pth    
  inflating: models/model_101.pth    
  inflating: models/model_109.pth    
  inflating: models/model_110.pth    
  inflating: models/model_105.pth    


In [3]:
!pip install pip install iterative-stratification

In [4]:
import torch
import torch.nn as nn
from torch.nn import functional as F
import torch.optim as optim
from torch.utils.data import DataLoader, Dataset
from torch.utils.data.dataset import random_split
from torch.optim.lr_scheduler import ReduceLROnPlateau

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import Normalizer
from sklearn.preprocessing import OneHotEncoder
from sklearn.model_selection import StratifiedKFold
from imblearn.over_sampling import RandomOverSampler

import numpy as np 
import pandas as pd 
from IPython.display import clear_output
import random 
import matplotlib.pyplot as plt
from scipy.ndimage.filters import gaussian_filter1d   ## smoother
from tqdm.notebook import tqdm, tnrange
import os
from iterstrat.ml_stratifiers import MultilabelStratifiedKFold

plt.rcParams['figure.figsize'] = 15, 7

CGREEN  = '\33[32m'
CBLUE =  '\033[34m'
CRED = '\033[1;31m'
CEND  = '\33[0m'

def seed_everything(seed=1903):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    print("seed kar diya")

/usr/local/lib/python3.6/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.neighbors. Anything that cannot be imported from sklearn.neighbors is now part of the private API.
  warnings.warn(message, FutureWarning)


In [5]:
if torch.cuda.is_available():
    device='cuda'
else:
    device='cpu'
    
device

'cuda'

In [6]:

train_features = pd.read_csv('train_features.csv')
train_targets = pd.read_csv('train_targets_scored.csv')
test_features = pd.read_csv('test_features.csv')


def preprocess(df):
    df = df.copy()
    df.loc[:, 'cp_type'] = df.loc[:, 'cp_type'].map({'trt_cp': 0, 'ctl_vehicle': 1})
    df.loc[:, 'cp_dose'] = df.loc[:, 'cp_dose'].map({'D1': 0, 'D2': 1})
    return df

train = preprocess(train_features)
test = preprocess(test_features)

del train_targets['sig_id']

target = train_targets.loc[train['cp_type']==0].reset_index(drop=True)
train = train.loc[train['cp_type']==0].reset_index(drop=True)

In [7]:

top_features = [  1,   2,   3,   4,   5,   6,   7,   9,  11,  14,  15,  16,  17,
        18,  19,  20,  21,  22,  23,  24,  25,  26,  27,  29,  30,  31,
        32,  33,  35,  36,  37,  38,  39,  40,  41,  42,  43,  44,  46,
        47,  48,  49,  50,  51,  52,  53,  54,  55,  56,  58,  59,  60,
        61,  62,  63,  64,  65,  66,  67,  68,  69,  70,  71,  72,  73,
        74,  75,  76,  78,  79,  80,  81,  82,  83,  84,  86,  87,  88,
        89,  90,  91,  92,  93,  94,  95,  96,  97,  98,  99, 100, 101,
       102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114,
       115, 116, 117, 118, 120, 121, 122, 123, 124, 125, 126, 127, 128,
       129, 130, 131, 132, 133, 136, 137, 138, 139, 140, 141, 142, 143,
       144, 145, 146, 147, 149, 150, 151, 152, 153, 154, 155, 156, 157,
       158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170,
       171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183,
       184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 197,
       198, 199, 200, 202, 203, 204, 205, 206, 208, 209, 210, 211, 212,
       213, 214, 215, 216, 217, 218, 219, 220, 221, 223, 224, 225, 226,
       227, 228, 229, 230, 231, 232, 233, 234, 235, 236, 237, 238, 239,
       240, 242, 243, 244, 245, 246, 247, 248, 249, 250, 251, 252, 253,
       254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266,
       267, 268, 269, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280,
       281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 294,
       295, 296, 298, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309,
       310, 311, 312, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323,
       324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336,
       337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349,
       350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362,
       363, 364, 365, 366, 367, 368, 369, 370, 371, 374, 375, 376, 377,
       378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 390, 391,
       392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404,
       405, 406, 407, 408, 409, 411, 412, 413, 414, 415, 416, 417, 418,
       419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431,
       432, 434, 435, 436, 437, 438, 439, 440, 442, 443, 444, 445, 446,
       447, 448, 449, 450, 453, 454, 456, 457, 458, 459, 460, 461, 462,
       463, 464, 465, 466, 467, 468, 469, 470, 471, 472, 473, 474, 475,
       476, 477, 478, 479, 481, 482, 483, 484, 485, 486, 487, 488, 489,
       490, 491, 492, 493, 494, 495, 496, 498, 500, 501, 502, 503, 505,
       506, 507, 509, 510, 511, 512, 513, 514, 515, 518, 519, 520, 521,
       522, 523, 524, 525, 526, 527, 528, 530, 531, 532, 534, 535, 536,
       538, 539, 540, 541, 542, 543, 544, 545, 546, 547, 549, 550, 551,
       552, 554, 557, 559, 560, 561, 562, 565, 566, 567, 568, 569, 570,
       571, 572, 573, 574, 575, 577, 578, 580, 581, 582, 583, 584, 585,
       586, 587, 588, 589, 590, 591, 592, 593, 594, 595, 596, 597, 599,
       600, 601, 602, 606, 607, 608, 609, 611, 612, 613, 615, 616, 617,
       618, 619, 620, 621, 622, 623, 624, 625, 626, 627, 628, 629, 630,
       631, 632, 633, 634, 635, 636, 637, 638, 639, 641, 642, 643, 644,
       645, 646, 647, 648, 649, 650, 651, 652, 654, 655, 656, 658, 659,
       660, 661, 662, 663, 664, 665, 666, 667, 668, 669, 670, 671, 672,
       673, 674, 675, 676, 677, 678, 679, 680, 681, 682, 683, 684, 685,
       686, 687, 688, 689, 691, 692, 693, 694, 695, 696, 697, 699, 700,
       701, 702, 704, 705, 707, 708, 709, 710, 711, 713, 714, 716, 717,
       718, 720, 721, 723, 724, 725, 726, 727, 728, 729, 730, 731, 732,
       733, 734, 735, 737, 738, 739, 740, 742, 743, 744, 745, 746, 747,
       748, 749, 750, 751, 752, 753, 754, 755, 756, 757, 759, 760, 761,
       762, 763, 764, 765, 766, 767, 768, 769, 770, 771, 772, 773, 774,
       775, 776, 777, 779, 780, 781, 782, 783, 784, 785, 786, 787, 788,
       789, 790, 792, 793, 794, 795, 796, 797, 798, 800, 801, 802, 803,
       804, 805, 806, 808, 809, 811, 813, 814, 815, 816, 817, 818, 819,
       821, 822, 823, 825, 826, 827, 828, 829, 830, 831, 832, 834, 835,
       837, 838, 839, 840, 841, 842, 845, 846, 847, 848, 850, 851, 852,
       854, 855, 856, 858, 859, 860, 861, 862, 864, 866, 867, 868, 869,
       870, 871, 872, 873, 874]

all_columns = train.columns
train=train[all_columns[top_features]]
test = test[all_columns[top_features]]

train = train.values
test = test.values
target = target.values

In [8]:

class TrainDataset(Dataset):
    def __init__(self, train,targets, noise ):
        
        self.features  = train
        self.targets = targets
        self.noise = noise
        
    def sizes(self):
        print("features size = ", self.features.shape[1])
        print("targets size = ", self.targets.shape[1])

        
    def __len__(self):
        return self.features.shape[0]

    def __getitem__(self, idx):
        feature = torch.tensor(self.features[idx]).float()
        
#         if self.noise == True:
# #             print("noisy boi")
#             feature  = feature + torch.randn_like(feature)/150
            
        target = torch.tensor(self.targets[idx]).float()
        
        return feature, target

In [9]:
class Model(nn.Module):
    def __init__(self):
        super(Model, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(785)
        self.dropout1 = nn.Dropout(0.2)
        self.dense1 = nn.utils.weight_norm(nn.Linear(785, 2048))
        
        self.batch_norm2 = nn.BatchNorm1d(2048)
        self.dropout2 = nn.Dropout(0.5)
        self.dense2 = nn.utils.weight_norm(nn.Linear(2048, 1048))
        
        self.batch_norm3 = nn.BatchNorm1d(1048)
        self.dropout3 = nn.Dropout(0.5)
        self.dense3 = nn.utils.weight_norm(nn.Linear(1048, 206))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        
        x = self.batch_norm2(x)
        x = self.dropout2(x)
        x = F.relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dropout3(x)
        x = self.dense3(x)
        
        return x

class Model_2(nn.Module):
    def __init__(self):
        super(Model_2, self).__init__()
        self.batch_norm1 = nn.BatchNorm1d(785)
        self.dropout1 = nn.Dropout(0.5)
        self.dense1 = nn.utils.weight_norm(nn.Linear(785, 790))
        
#         self.batch_norm2 = nn.BatchNorm1d(900)
#         self.dropout2 = nn.Dropout(0.5)
#         self.dense2 = nn.utils.weight_norm(nn.Linear(900, 900))
        
        self.batch_norm3 = nn.BatchNorm1d(790)
        self.dense3 = nn.utils.weight_norm(nn.Linear(790, 206))
    
    def forward(self, x):
        x = self.batch_norm1(x)
        x = self.dropout1(x)
        x = F.relu(self.dense1(x))
        
#         x = self.batch_norm2(x)
#         x = self.dropout2(x)
#         x = F.leaky_relu(self.dense2(x))
        
        x = self.batch_norm3(x)
        x = self.dense3(x)
        
        return x

In [10]:
class pytorch_model():
    def __init__(self, model_class, model_path, device):
        
        self.model_class = model_class
        self.model_path = model_path
        self.device = device
        
        self.model = self.model_class()
        self.model.load_state_dict(torch.load(self.model_path))
        self.model.to(self.device)
        self.model.eval()
        
    def predict(self, x):
        
        pred = self.model(x)
        
        return pred 
    
    def __call__(self, x):
        return self.predict(x)




In [11]:
class blend(nn.Module):
    def __init__(self,weights, model_list ,device = device):
        super(blend, self).__init__()
        self.model_list = model_list
        self.weights = weights 


    def forward(self, x):
        
        x_list = [self.model_list[i](x).detach().cpu() for i in range(len(self.model_list))]
        
        final_pred = torch.zeros_like(x_list[0])
        
        for i in range(len(x_list)):
            final_pred += x_list[i] * self.weights[i]
        final_pred = final_pred/self.weights.sum()
        
        return final_pred.cpu()
    
    def __call__(self,x):
        return self.forward(x)

def inference_fn(model, test_features, device):

    results = np.zeros([test_features.shape[0], 206])
    test_features_tensor = torch.tensor(test_features).float().to(device)
    rows = model(test_features_tensor).sigmoid().detach().cpu().numpy()
    results = rows 

    return results 


def log_loss_metric(y_true, y_pred):
    y_pred_clip = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = - np.mean(np.mean(y_true * np.log(y_pred_clip) + (1 - y_true) * np.log(1 - y_pred_clip), axis = 1))
    return loss


def create_weights_from_val_losses(val_losses_np):
    w = 1/val_losses_np
    w_norm = w/w.max()
    return w_norm

In [12]:
train_main = train[:-1000]
target_main = target[:-1000]

train_hold = train[-1000:]
target_hold = target[-1000:]

hold_dataset = TrainDataset(train_hold, target_hold, noise = False)
hold_loader = DataLoader(dataset=hold_dataset, batch_size=512, shuffle= False)

num_folds = 10

mskf = MultilabelStratifiedKFold(n_splits=num_folds, shuffle=True, random_state=0)

train_loaders = []
val_loaders = []

for k , (train_idx,valid_idx) in enumerate(mskf.split(train_main,target_main)):

    x_train,x_valid,y_train,y_valid = train_main[train_idx,:],train_main[valid_idx,:],target_main[train_idx,:],target_main[valid_idx,:]

    input_size = x_train.shape[1]
    output_size = target.shape[1]


    train_dataset = TrainDataset(x_train, y_train, noise = False)
    valid_dataset = TrainDataset(x_valid, y_valid, noise = False)

    train_loader = DataLoader(dataset=train_dataset, batch_size=128, shuffle=True)

    val_loader = DataLoader(dataset=valid_dataset, batch_size=256, shuffle = False)
    
    train_loaders.append(train_loader)
    val_loaders.append(val_loader)

print(len(train_loader), "batches ")
print(len(val_loader), " batches ")

148 batches 
9  batches 


# Train non nn models 

In [15]:
import gc
import datetime
from sklearn.kernel_approximation import Nystroem
from sklearn.isotonic import IsotonicRegression
from sklearn.linear_model import LogisticRegression
# from cuml import LogisticRegression
from sklearn.kernel_ridge import KernelRidge
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import log_loss
from tqdm.notebook import tqdm
from time import time
import sys

def log_loss_metric(y_true, y_pred):
    y_pred_clip = np.clip(y_pred, 1e-15, 1 - 1e-15)
    loss = - np.mean(np.mean(y_true * np.log(y_pred_clip) + (1 - y_true) * np.log(1 - y_pred_clip), axis = 1))
    return loss

In [16]:
num_folds = 2

kernel_ridge_models = []

mskf = MultilabelStratifiedKFold(n_splits=num_folds, shuffle=True, random_state=0)

train_loaders = []
val_loaders = []

for k , (train_idx,valid_idx) in enumerate(mskf.split(train_main,target_main)):

    start_time = time()

    x_train,x_valid,y_train,y_valid = train_main[train_idx,:],train_main[valid_idx,:],target_main[train_idx,:],target_main[valid_idx,:]

    model = KernelRidge(alpha = 80, kernel = 'rbf')

    model.fit(x_train, y_train)

    fold_pred = model.predict(x_valid)
    fold_score = log_loss_metric(y_valid, fold_pred)
    
    kernel_ridge_models.append(model)
    print(f'[{str(datetime.timedelta(seconds = time() - start_time))[2:7]}] KRR: Fold {k}:', fold_score)

[00:26] KRR: Fold 0: 0.021886446741176516
[00:26] KRR: Fold 1: 0.021801946600989074


In [17]:
class k_r_model():
    def __init__(self, model, mode = "numpy"):
        
        self.mode = mode
        self.model = model
        
    def forward(self, x):
        
        if self.mode == "torch":

            pred = self.model.predict(x.cpu().numpy())
            pred = torch.tensor(pred)
            
        elif self.mode == "numpy":

            pred = self.model.predict(x)

        return pred
    
    def __call__(self, x):
        return self.forward(x)

In [18]:
fold_pred = kernel_ridge_models[0].predict(train_hold)
fold_score = log_loss_metric(target_hold, fold_pred)
fold_score

0.02162352251011992

In [19]:
start = "/content/models/model_"
names = [start + str(i +1) + ".pth" for i in range(10)]
names_2 = [start + str(i +100 +1 ) + ".pth" for i in range(10)]
names.extend(names_2)

all_models = []

for i in range(len(names)):
    if i < 9:
        model_class = Model
    else:
        model_class = Model_2
    m = pytorch_model(model_class, names[i], device = device)
    
    all_models.append(m)
    


In [20]:
for i in range(len(kernel_ridge_models)):
    temp_k_r_model = k_r_model(model, mode = "torch")
    all_models.append(temp_k_r_model)



In [ ]:
all_models

In [22]:
poop_weights = np.ones(25)

blend_boi = blend(poop_weights, all_models)

sample_infer = inference_fn(blend_boi, train_hold, device)
loss = log_loss_metric(target_hold, sample_infer)
loss

0.017932491935568572

In [ ]:
!pip install optuna

In [24]:
import optuna   

In [27]:
def find_loss(params):
    
    weights_np = np.array(list(params.values()))
    blend_boi = blend(weights_np, all_models)
    sample_infer = inference_fn(blend_boi, train_hold, device)
    loss = log_loss_metric(target_hold, sample_infer)
    
    return loss


def objective(trial):

    params = {
        "w1": trial.suggest_uniform("w1", 0.01, 1.0),
        "w2": trial.suggest_uniform("w2", 0.01, 1.0),
        "w3": trial.suggest_uniform("w3", 0.01, 1.0),
        "w4": trial.suggest_uniform("w4", 0.01, 1.0),
        "w5": trial.suggest_uniform("w5", 0.01, 1.0),
        "w6": trial.suggest_uniform("w6", 0.01, 1.0),
        "w7": trial.suggest_uniform("w7", 0.01, 1.0),
        "w8": trial.suggest_uniform("w8", 0.01, 1.0),
        "w9": trial.suggest_uniform("w9", 0.01, 1.0),
        "w10": trial.suggest_uniform("w10", 0.01, 1.0),
        "w11": trial.suggest_uniform("w11", 0.01, 1.0),
        "w12": trial.suggest_uniform("w12", 0.01, 1.0),
        "w13": trial.suggest_uniform("w13", 0.01, 1.0),
        "w14": trial.suggest_uniform("w14", 0.01, 1.0),
        "w15": trial.suggest_uniform("w15", 0.01, 1.0),
        "w16": trial.suggest_uniform("w16", 0.01, 1.0),
        "w17": trial.suggest_uniform("w17", 0.01, 1.0),
        "w18": trial.suggest_uniform("w18", 0.01, 1.0),
        "w19": trial.suggest_uniform("w19", 0.01, 1.0),
        "w20": trial.suggest_uniform("w20", 0.01, 1.0),
        "w21": trial.suggest_uniform("w21", 0.01, 1.0),
        "w22": trial.suggest_uniform("w22", 0.01, 1.0)
        # "w23": trial.suggest_uniform("w23", 0.01, 1.0),
        # "w24": trial.suggest_uniform("w24", 0.01, 1.0),
        # "w25": trial.suggest_uniform("w25", 0.01, 1.0)

    }

    loss_ = find_loss(params) 
    return loss_

In [28]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=1000)


[I 2020-10-03 17:41:33,519] A new study created in memory with name: no-name-6e87b312-142b-404f-9b82-f9829fd50398
[I 2020-10-03 17:41:35,189] Trial 0 finished with value: 0.01593326906418095 and parameters: {'w1': 0.7926197307897748, 'w2': 0.12498351206176805, 'w3': 0.8032232426597569, 'w4': 0.3192833985512568, 'w5': 0.2701802542719369, 'w6': 0.8125591039126907, 'w7': 0.3094643786050861, 'w8': 0.228016759696843, 'w9': 0.0313335426553996, 'w10': 0.8342566685107502, 'w11': 0.6946833833314185, 'w12': 0.5053157605441496, 'w13': 0.6410864346538635, 'w14': 0.5285012841105888, 'w15': 0.46632520288564805, 'w16': 0.22241434967672385, 'w17': 0.7272405381106114, 'w18': 0.29953735571455664, 'w19': 0.3816261736710958, 'w20': 0.4900712527030898, 'w21': 0.11748684755782896, 'w22': 0.7176316100437249}. Best is trial 0 with value: 0.01593326906418095.
[I 2020-10-03 17:41:36,885] Trial 1 finished with value: 0.015928520851705253 and parameters: {'w1': 0.46255983423697467, 'w2': 0.6183497337789106, 'w3':

KeyboardInterrupt: ignored